In [13]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import talib as ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from pandas.tseries.offsets import DateOffset
from sklearn import metrics
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# the pandas webreader provides remote data access to apis
import pandas_datareader as webreader

date_today = "2022-11-10"
date_start = "2010-01-01"

# set the symbol to bitcoin-usd quotes
# for more symbols check yahoo.finance.com
symbol = "BTC-USD"

In [3]:
# now we will send the request to the yahoo finance api endpoint
df = webreader.DataReader(symbol, start=date_start, end=date_today, data_source="yahoo")
df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2014-09-17,468.174011,452.421997,465.864014,457.334015,21056800,457.334015
2014-09-18,456.859985,413.104004,456.859985,424.440002,34483200,424.440002
2014-09-19,427.834991,384.532013,424.102997,394.795990,37919700,394.795990
2014-09-20,423.295990,389.882996,394.673004,408.903992,36863600,408.903992
2014-09-21,412.425995,393.181000,408.084991,398.821014,26580100,398.821014


In [4]:
df['Pct Change'] = df['Close'].pct_change()

In [5]:
df['50 SMA'] = ta.SMA(df['Close'], timeperiod=50)
df['200 SMA'] = ta.SMA(df['Close'], timeperiod=200)
df['10 EMA'] = ta.EMA(df['Close'], timeperiod=10)
df['20 EMA'] = ta.EMA(df['Close'], timeperiod=20)
df['RSI'] = ta.RSI(df['Close'], timeperiod=14)


In [6]:
#Create variable to predict 'n' days into the future
projection = 14

df['prediction'] = df[['Close']].shift(-projection)
df

,High,Low,Open,Close,Volume,Adj Close,Pct Change,50 SMA,200 SMA,10 EMA,20 EMA,RSI,prediction
Date,,,,,,,,,,,,,
2014-09-17,468.174011,452.421997,465.864014,457.334015,21056800,457.334015,NaN,NaN,NaN,NaN,NaN,NaN,383.614990
2014-09-18,456.859985,413.104004,456.859985,424.440002,34483200,424.440002,-0.071926,NaN,NaN,NaN,NaN,NaN,375.071991
2014-09-19,427.834991,384.532013,424.102997,394.795990,37919700,394.795990,-0.069843,NaN,NaN,NaN,NaN,NaN,359.511993
2014-09-20,423.295990,389.882996,394.673004,408.903992,36863600,408.903992,0.035735,NaN,NaN,NaN,NaN,NaN,328.865997
2014-09-21,412.425995,393.181000,408.084991,398.821014,26580100,398.821014,-0.024659,NaN,NaN,NaN,NaN,NaN,320.510010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07,21053.246094,20489.972656,20924.621094,20602.816406,53510852236,20602.816406,-0.015467,19683.553633,23881.031875,20634.032692,20341.873001,55.038707,NaN
2022-11-08,20664.607422,17603.544922,20600.671875,18541.271484,118992465607,18541.271484,-0.100061,19663.496484,23775.036631,20253.530654,20170.387142,35.913352,NaN
2022-11-09,18590.458984,15682.692383,18543.761719,15880.780273,102905151606,15880.780273,-0.143490,19603.296309,23657.006880,19458.485130,19761.853155,24.217638,NaN


In [9]:
#create independent data set (X)
X = np.array(df[['Close']])
#Remove last 14 rows of data because of nan
X = X[:-projection]
print(X)

[[  457.33401489]
 [  424.44000244]
 [  394.79598999]
 ...
 [20770.44140625]
 [20285.8359375 ]
 [20595.3515625 ]]


In [10]:
#create dependent dataset (y)
y =df['prediction'].values
y = y[:-projection]
print(y)

[  383.61499023   375.07199097   359.51199341 ... 15880.78027344
 17586.77148438 17034.29296875]


In [11]:
#split the data into 85% training and 15% testing datasets
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size= .15)

In [14]:
#create and train the model
linReg = LinearRegression()
#Train the model
linReg.fit(x_train, y_train)

LinearRegression()

In [23]:
scaler = StandardScaler()
X_scaler = scaler.fit(x_train)
X_train_scaled = X_scaler.transform(x_train)
X_test_scaled = X_scaler.transform(x_test)

In [24]:
#Test the model using score
linReg_confidence = linReg.score(x_test,y_test)
print('Linear regression Confidence:', linReg_confidence)

Linear regression Confidence: 0.9683443297743748


In [25]:
#Create a variable called x_projection and set equal to last 14 rows of data from original dataset.
x_projection = np.array(df[['Close']])[-projection:]
print(x_projection)

[[20818.4765625 ]
 [20635.60351562]
 [20495.7734375 ]
 [20485.2734375 ]
 [20159.50390625]
 [20209.98828125]
 [21147.23046875]
 [21282.69140625]
 [20926.48632812]
 [20602.81640625]
 [18541.27148438]
 [15880.78027344]
 [17586.77148438]
 [17034.29296875]]


In [26]:
#Print the linear regression models predictions for next 14 days
lin_Reg_prediction = linReg.predict(x_projection)
print(lin_Reg_prediction)

[20724.08222756 20544.96033407 20407.99849749 20397.71387702
 20078.62663986 20128.07546239 21046.09262321 21178.77494044
 20829.87646057 20512.8457649  18493.58794518 15887.66962394
 17558.66696991 17017.52107924]
